In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset
import random
import time

DATA_DIR = "data/animals/"
BATCH_SIZE = 32
EPOCHS = 10
IMG_SIZE = 128
LR = 0.001
NUMBER_OF_IMAGES = 250
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [85]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
class_names = dataset.classes

class_indices = {i: [] for i in range(len(class_names))}

for idx, (_, label) in enumerate(dataset):
    class_indices[label].append(idx)

selected_indices = []
for label, indices in class_indices.items():
    random.shuffle(indices)
    selected_indices.extend(indices[:NUMBER_OF_IMAGES])

dataset = Subset(dataset, selected_indices)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

In [86]:
class CnnVgg16(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (IMG_SIZE//8) * (IMG_SIZE//8), 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [87]:
model = CnnVgg16().to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

def accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(images)
            preds = torch.sigmoid(outputs) > 0.5
            preds = preds.squeeze().int()

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total

In [88]:
start_time = time.time()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(DEVICE)
        labels = labels.float().unsqueeze(1).to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}]: "
          f"Loss: {total_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_acc:.3f}, "
          f"Test Accuracy: {test_acc:.3f}")

total_time = time.time() - start_time
print(f"\nTotal training time: {total_time:.2f} seconds")

torch.save(model.state_dict(), "saved_models/cat_dog_cnn.pt")
print("Model saved as saved_models/cat_dog_cnn.pt")

Epoch [1/10]: Loss: 0.7001, Train Accuracy: 0.530, Test Accuracy: 0.580
Epoch [2/10]: Loss: 0.6887, Train Accuracy: 0.532, Test Accuracy: 0.570
Epoch [3/10]: Loss: 0.6943, Train Accuracy: 0.515, Test Accuracy: 0.520
Epoch [4/10]: Loss: 0.6935, Train Accuracy: 0.583, Test Accuracy: 0.600
Epoch [5/10]: Loss: 0.6688, Train Accuracy: 0.605, Test Accuracy: 0.600
Epoch [6/10]: Loss: 0.6889, Train Accuracy: 0.545, Test Accuracy: 0.550
Epoch [7/10]: Loss: 0.6644, Train Accuracy: 0.502, Test Accuracy: 0.490
Epoch [8/10]: Loss: 0.6689, Train Accuracy: 0.557, Test Accuracy: 0.560
Epoch [9/10]: Loss: 0.6614, Train Accuracy: 0.595, Test Accuracy: 0.540
Epoch [10/10]: Loss: 0.6381, Train Accuracy: 0.598, Test Accuracy: 0.540

Total training time: 125.17 seconds
Model saved as saved_models/cat_dog_cnn.pt
